In [2]:
import pandas as pd
import time
import random

%matplotlib inline

/Users/um003282/.pyenv/versions/3.5.2/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/Users/um003282/.pyenv/versions/3.5.2/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [43]:
reader=pd.read_csv("../output/reduced_merged_removeT.csv.gz", compression="gzip", chunksize=10000)

TypeError: Population must be a sequence or set.  For dicts, use list(d).